# 🌦️ WeatherWise – Built by Rudra

Welcome to **WeatherWise**, a smart weather advisor app crafted by Rudra. This project leverages Python, data visualization, and AI tools to transform weather data into insightful and actionable guidance. Explore the power of intelligent forecasting with WeatherWise!


## 🧰 Setup and Imports


In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus


In [ ]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

## 📦 Setup and Configuration


In [15]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
import re
import json
import numpy as np
# ✅ Import after installing
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

## 🌤️ Weather Data Functions

In [53]:
# function to retrive weather data
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast, or an error message
    """
    # Input validation
    if not isinstance(location, str) or not location.strip():
        return {"error": "Invalid location. Please provide a non-empty city name."}
    if not isinstance(forecast_days, int) or not (1 <= forecast_days <= 5):
        return {"error": "Forecast days must be an integer between 1 and 5."}

    try:
        weather = get_weather(location=location, view_options=str(forecast_days), with_metadata=False)
        parsed_data = json.loads(weather)

        current_condition = parsed_data.get("current_condition", [{}])[0]
        weather_forecast = parsed_data.get("weather", [])[:forecast_days]

        # Destructure and keep the relevent data from the response
        result = {
            "location": location.title(),
            "current": {
                "temperature_C": current_condition.get("temp_C"),
                "weather_desc": current_condition.get("weatherDesc", [{}])[0].get("value"),
                "humidity": current_condition.get("humidity"),
                "wind_kph": current_condition.get("windspeedKmph"),
                "feels_like_C": current_condition.get("FeelsLikeC")
            },
            "forecast": []
        }

        for day in weather_forecast:
            result["forecast"].append({
                "date": day.get("date"),
                "max_temp_C": day.get("maxtempC"),
                "max_temp_F": day.get("maxtempF"),
                "min_temp_C": day.get("mintempC"),
                "min_temp_F": day.get("mintempF"),
                "avg_temp_C": day.get("avgtempC"),
                "avg_temp_F": day.get("avgtempF"),
                "sun_hour": day.get("sunHour"),
                "total_Snow_cm": day.get("totalSnow_cm"),
                "uv_index": day.get("uvIndex"),
                "description": day.get("hourly", [{}])[0].get("weatherDesc", [{}])[0].get("value")
            })

        return result

    except Exception as e:
        return {"error": f"Failed to retrieve weather data: {str(e)}"}


## 📊 Visualisation Functions

In [11]:
def temperature_visualisation(weather_data):
    dates = [f['date'] for f in weather_data['forecast']]
    min_temps = [f['min_temp_C'] for f in weather_data['forecast']]
    avg_temps = [f['avg_temp_C'] for f in weather_data['forecast']]
    max_temps = [f['max_temp_C'] for f in weather_data['forecast']]

    plt.figure(figsize=(8, 5))
    plt.plot(dates, min_temps, marker='o', label='Min Temp (°C)', color='skyblue')
    plt.plot(dates, avg_temps, marker='o', label='Avg Temp (°C)', color='orange')
    plt.plot(dates, max_temps, marker='o', label='Max Temp (°C)', color='red')

    plt.title(f"Daily Temperature Trends in {weather_data['location']}")
    plt.xlabel("Date")
    plt.ylabel("Temperature (°C)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [13]:

def sun_uv_visualisation(weather_data, output_type='display'):
    dates = [f['date'] for f in weather_data['forecast']]
    sun_hours = [f['sun_hour'] for f in weather_data['forecast']]
    uv_indices = [f['uv_index'] for f in weather_data['forecast']]

    x = np.arange(len(dates))
    width = 0.35

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(x - width/2, sun_hours, width, label='Sun Hours', color='gold')
    ax.bar(x + width/2, uv_indices, width, label='UV Index', color='purple')

    ax.set_title(f"Sunlight and UV Index Forecast in {weather_data['location']}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Value")
    ax.set_xticks(x)
    ax.set_xticklabels(dates)
    ax.legend()
    ax.grid(True, axis='y')
    plt.tight_layout()
    plt.show()

## 🤖 Natural Language Processing

In [54]:
import spacy

nlp = spacy.load("en_core_web_sm")

TIME_PERIODS = {
    "now", "right now", "currently", "today", "tonight", "this morning", "this afternoon", "this evening",
    "tomorrow", "tomorrow morning", "tomorrow afternoon", "tomorrow evening",
    "day after tomorrow", "the day after tomorrow",
    "weekend", "this weekend", "next weekend",
    "next week", "this week",
    "next month", "this month",
    "in an hour", "in 1 hour", "in 2 hours", "in 3 hours",
    "hourly", "daily", "weekly", "monthly"
}

WEATHER_ATTRIBUTES = {
    "temperature", "rain", "rainfall", "humidity", "wind", "wind speed",
    "snow", "snowfall", "sunshine", "sun", "cloud", "cloudiness", "uv index", "pressure"
}

def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    question_lower = question.lower()
    doc = nlp(question_lower)

    locations = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
    time_periods = [tp for tp in TIME_PERIODS if tp in question_lower]
    attributes = [attr for attr in WEATHER_ATTRIBUTES if attr in question_lower]

    return {
        "locations": locations,
        "time_periods": time_periods,
        "weather_attributes": attributes
    }

## 🧩 Main Application Logic

In [55]:
# Tie everything together here
def generate_weather_response(question):
    """
    Handle a weather-related question and return the weather data.

    Args:
        question (str): The user's natural language weather question.

    Returns:
        dict: Weather information or an error message.
    """
    parsed = parse_weather_question(question)

    locations = parsed.get("locations")
    time_periods = parsed.get("time_periods", [])

    # Fallback if no location is found
    if not locations:
        return {"error": "Could not identify a location in your question."}

    location = locations[0]

    # Determine forecast days based on time_periods
    forecast_days = 1  # default: today only
    if any(tp in time_periods for tp in ["tomorrow", "tomorrow morning", "tomorrow afternoon", "tomorrow evening"]):
        forecast_days = 2
    elif any(tp in time_periods for tp in ["day after tomorrow", "the day after tomorrow"]):
        forecast_days = 3
    elif any(tp in time_periods for tp in ["this week", "next week", "weekend", "next weekend"]):
        forecast_days = 5

    return {
        "parsed": parsed,
        "weather_data": get_weather_data(location, forecast_days=forecast_days)
    }

## 🧭 User Interface

In [ ]:
def handle_weather_query():
  while 1:
    question = pyip.inputStr(prompt="Your Query:", blank=True)
    if question == 'exit':
      break
    parsed_data = generate_weather_response(question)
    response = get_response(f"Generate answer in natural language where data is {parsed_data}")
    print(response)

## 🧪 Testing and Examples

In [ ]:
def test_get_weather_io():
    """
    Demonstrates a sample input and output for the get_weather function.
    """
    location = input("Sample Input:")
    forecast_days = int(input("Forecast Days:"))

    print("Sample Output:")
    weather_data = get_weather_data(location, forecast_days)

    if weather_data:
        from pprint import pprint
        pprint(weather_data)
    else:
        print("Failed to retrieve or parse weather data.")

test_get_weather_io()

In [ ]:
def test_data_visulaization_io():
    """
    Demonstrates a sample input and output for the get_weather function.
    """
    location = input("Sample Input:")
    forecast_days = int(input("Forecast Days:"))

    print("Sample Output:")
    weather_data = get_weather_data(location, forecast_days)

    if weather_data:
        temperature_visualisation(weather_data)
        sun_uv_visualisation(weather_data)
    else:
        print("Failed to retrieve or parse weather data.")

test_data_visulaization_io()

In [ ]:
# test_parse_weather_question
query = "Will it rain this weekend in New York or next week in San Francisco?"
result = parse_weather_question(query)
print(result)

In [ ]:
# test AI chat
# type 'exit' in the input to terminate the process
handle_weather_query()